# Use example of visturing: PerceptNet

This is a simple example where we use the NLPD distance and evaluate it in all the experiments proposed in:
"A Turing Test for Artificial Nets devoted to Vision"
https://www.frontiersin.org/journals/artificial-intelligence/articles/10.3389/frai.2025.1665874/abstract


In [ ]:
# If the toolbox is not installed...
#!pip install git+https://github.com/Jorgvt/visturing.git

  Cloning https://github.com/Jorgvt/visturing.git to /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-req-build-04yee95_
  Running command git clone --filter=blob:none --quiet https://github.com/Jorgvt/visturing.git /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-req-build-04yee95_
  Resolved https://github.com/Jorgvt/visturing.git to commit 0680dbedabd4d7e864f8355240ba841000239497
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl (5.4 

In [1]:
from visturing.properties import evaluate_all

## Define the metric

In [2]:
!pip install git+https://github.com/Jorgvt/paramperceptnet

  Cloning https://github.com/Jorgvt/paramperceptnet to /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-req-build-7n35p6_c
  Running command git clone --filter=blob:none --quiet https://github.com/Jorgvt/paramperceptnet /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-req-build-7n35p6_c
  Resolved https://github.com/Jorgvt/paramperceptnet to commit f0e727271d2abeab34eeec76c66368eb705d74e4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/Jorgvt/fxlayers.git to /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-install-pysp63n7/fxlayers_91676be9da844c7db11eff9adf2d36f0
  Running command git clone --filter=blob:none --quiet https://github.com/Jorgvt/fxlayers.git /private/var/folders/4s/2yd8m6kx4zn8143hhnlkqm600000gn/T/pip-install-pysp63n7/fxlayers_91676be9da844c7db11eff9adf2d36f0
  Resolved https://github.com/Jorgvt/fxlayers.git to commit 301

In [3]:
import os
# Forzar a JAX a usar la CPU y evitar el error de Metal
os.environ["JAX_PLATFORM_NAME"] = "cpu"

import json

import jax
from jax import random, numpy as jnp
import flax
from huggingface_hub import hf_hub_download
from ml_collections import ConfigDict

from paramperceptnet.models import PerceptNet
from paramperceptnet.configs import param_config

Metal device set to: Apple M2 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



W0000 00:00:1764864392.600478  719710 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1764864392.610191  719710 service.cc:145] XLA service 0x303fcb360 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764864392.610200  719710 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1764864392.611308  719710 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1764864392.611316  719710 mps_client.cc:384] XLA backend will use up to 22906109952 bytes on device 0 for SimpleAllocator.


In [4]:
model_name = "ppnet-fully-trained"
config_path = hf_hub_download(repo_id=f"Jorgvt/{model_name}",
                              filename="config.json")
with open(config_path, "r") as f:
    config = ConfigDict(json.load(f))


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

In [5]:
from safetensors.flax import load_file

weights_path = hf_hub_download(repo_id=f"Jorgvt/{model_name}",
                               filename="weights.safetensors")
variables = load_file(weights_path)
variables = flax.traverse_util.unflatten_dict(variables, sep=".")
state = variables["state"]
params = variables["params"]

weights.safetensors:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

In [ ]:
model = PerceptNet(config)


def l2_dist(a, b):
    return ((a-b)**2).mean(axis=[1,2,3])**(1/2)

In [ ]:
# The distance function shoud work as 
# input: two batches of images (NxHxWxC)
# output: one distance value per image (N) 

import numpy as np


def evaluate_PerceptNet(ImA,ImB):
    # Imagenes dimensiones (Nims,H,W,C)
    # Normalizadas a [0,255]
    image_A = (ImA/255)
    image_B = (ImB/255)
    
    distances = np.zeros(ImA.shape[0])

    for i in range(ImA.shape[0]):
        pred = model.apply({"params": params, **state}, image_A[i:i+1], train=False)
        pred_dist = model.apply({"params": params, **state}, image_B[0:1], train=False)
        distances[i] = l2_dist(pred, pred_dist)[0]

    return distances

## Validate Metric Suitability for Experiments

In [22]:
# Testing the function behaviour
ImA = np.abs(255*np.random.rand(5,128,128,3))
ImB = np.abs(255*np.random.rand(1,128,128,3))

evaluate_PerceptNet(ImA,ImB)

array([0.06235121, 0.06501545, 0.06609842, 0.06670254, 0.06390374])

## Perform the experiments

In [24]:
# This function evaluates all the experiments proposed in the paper
# The first time it downloads the data, so it will take long
# If the data is already downloaded it should take a couple of seconds

results = evaluate_all(evaluate_PerceptNet,
             data_path="./Data/",
             gt_path="./Data/")

prop1 done
prop2 done
prop3_4 done


/Users/valero/miniconda3/lib/python3.11/site-packages/visturing/properties/prop5.py:65: RuntimeWarning: divide by zero encountered in divide
  diffs_inv = {k:v/diffs_a for k, v in diffs.items()}


prop5 done
prop6_7 done
prop8 done
prop9 done
prop10 done


## Results

In [25]:
print(results['prop1']['corr'])
print(results['prop2']['correlations']['pearson_achrom'])
print(results['prop2']['correlations']['pearson_chrom'])
print(results['prop3_4']['pearson_corr'][0])
print(results['prop5']['person_corr'][0])
print(results['prop6_7']['pearson'][0])
print(results['prop8']['pearson'][0])

0.5678528556337681
0.9830485147650178
-0.3264891180214861
0.6536564756373772
-0.35102403854566455
0.02130087192241272
0.3244076720514194
